In [25]:
import pandas as pd
from collections import defaultdict
import re
import numpy as np
import sys
sys.path.insert(0, '../../base_codes')
sys.path.insert(0, '../../data_specific_codes')
sys.path.insert(0, '../../utility_codes')

#%run -i 'parameters'
#%run -i 'main_functions'
#%run -i 'utility_functions'

from RE_init import *
from main_functions import *
from utility_functions import *

##ALL the books 
pronoun_prefix = "_raw" # no pronoun: "_raw" - with pronoun: "0601"
file_input_postfix = "_with_SVP_relations_-1.csv" 
file_output_postfix = "_relations_-1.csv"
svp_prefix = "_with_conjAND"#"_with_SVP"

##MICE_AND_MEN
#based_dir = '../../../../data/GoodReads/revised_pronoun_1Jun2017/results/Mice_and_men/results_with_svp_relationships/'
#based_dir = '../../../../data/GoodReads/Raw_data_no_pronoun/mice_and_men/results_with_svp_relationships/'
#file_input_name = "mice_and_men" + pronoun_prefix + file_input_postfix
#input_name_prefix = 'mice_and_men' + pronoun_prefix + svp_prefix
#DATA_SET = 'goodreads-Mice-and-men'

##HOBBIT
#based_dir = '../../../../data/GoodReads/revised_pronoun_1Jun2017/results/Hobbit/results_with_svp_relationships/'
#based_dir = '../../../../data/GoodReads/Raw_data_no_pronoun/hobbit/results_with_svp_relationships/'
#file_input_name= 'hobbit' + pronoun_prefix + file_input_postfix
#input_name_prefix = 'hobbit' + pronoun_prefix + svp_prefix
#DATA_SET = 'goodreads-Hobbit'

##Mockingbird
#based_dir = '../../../../data/GoodReads/revised_pronoun_1Jun2017/results/Mockingbird/results_with_svp_relationships/'
#based_dir = '../../../../data/GoodReads/Raw_data_no_pronoun/mockingbird/results_with_svp_relationships/'
#file_input_name= 'mockingbird' + pronoun_prefix + file_input_postfix
#input_name_prefix = 'mockingbird' + pronoun_prefix + svp_prefix
#DATA_SET = 'goodreads-Mockingbird'

##Frankenstein
#based_dir = '../../../../data/GoodReads/revised_pronoun_1Jun2017/results/Frankenstein/results_with_svp_relationships/'
based_dir = '../../../../data/GoodReads/Raw_data_no_pronoun/frankenstein/results_with_svp_relationships/'
file_input_name= 'frankenstein' + pronoun_prefix + file_input_postfix
input_name_prefix = 'frankenstein' + pronoun_prefix + svp_prefix
DATA_SET = 'goodreads-Frankenstein'

def read_df_rel(based_dir, file_input_name):
    file_input = based_dir + file_input_name    
    ff = open(file_input)
    delim=","
    df = pd.read_csv(file_input,delimiter=delim,header=0)        
    return df

df_rels = read_df_rel(based_dir, file_input_name)

In [26]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [27]:
#pd.set_option('display.max_colwidth', -1)
#df_rels = df_rels.replace(np.nan, '', regex=True)
df_rels

,Unnamed: 0,post_num,sentence_num,sentence,arg1,rel,arg2,type,pattern,arg1_with_pos,rel_with_pos,arg2_with_pos,arg1_prepositions,rel_prepositions,arg2_prepositions,annotation
0,0,0,4,It's been almost 30 years since I've detested ...,{I},'ve {detested},this {much},SVO,"(nsubj, verb, dobj)",I-PRP-8,'ve-VBP-9 detested-VBN-10,this-DT-13 much-JJ-14,NaN,NaN,NaN,"{'dep_parse': ""nsubjpass(years-6, It-1)\nauxpa..."
1,1,0,7,"This time, I do not have the excuse that I was...",{I},do not {have},the {excuse},SVO,"(nsubj, verb, dobj)",I-PRP-4,do-VBP-5 not-RB-6 have-VB-7,the-DT-8 excuse-NN-9,NaN,NaN,NaN,"{'dep_parse': 'det(time-2, This-1)\nnsubj(have..."
2,2,0,7,"This time, I do not have the excuse that I was...",This {time},do not {have},the {excuse},SVO,"(nsubj, verb, dobj)",This-DT-1 time-NN-2,do-VBP-5 not-RB-6 have-VB-7,the-DT-8 excuse-NN-9,NaN,NaN,NaN,"{'dep_parse': 'det(time-2, This-1)\nnsubj(have..."
3,3,0,14,"OK, to be fair, I need to tell you what I like...",{I},{liked},{what},SVO,"(nsubj, verb, dobj)",I-PRP-13,liked-VBD-14,what-WP-12,NaN,REASON: prep_about TEXT: {this} --,NaN,"{'dep_parse': 'advmod(need-8, OK-1)\naux(fair-..."
4,4,0,14,"OK, to be fair, I need to tell you what I like...",{I},{liked} <<{what}>> about,{this},SV(O)P,"(nsubj, verb, (O)prep)",I-PRP-13,liked-VBD-14,NaN,NaN,NaN,NaN,"{'dep_parse': 'advmod(need-8, OK-1)\naux(fair-..."
5,5,0,15,"Well, Mary Shelley was a teen when she wrote t...",{she},when {wrote},{this},SVO,"(nsubj, verb, dobj)",she-PRP-9,when-WRB-8 wrote-VBD-10,this-DT-11,NaN,NaN,NaN,"{'dep_parse': 'advmod(teen-7, Well-1)\nnn(Shel..."
6,6,0,17,At 19 I was just looking for my next college b...,{I},just not {penning},the great English {classic},SVO,"(nsubj, verb, dobj)",I-PRP-3,just-RB-5 not-RB-13 penning-VBG-14,the-DT-15 great-JJ-16 English-JJ-17 classic-NN-18,NaN,REASON: prep_at TEXT: {19} -- REASON: prep_for...,NaN,"{'dep_parse': 'prep_at(penning-14, 19-2)\nnsub..."
7,7,0,17,At 19 I was just looking for my next college b...,{I},just not {penning} <<the great English {classi...,{19},SV(O)P,"(nsubj, verb, (O)prep)",I-PRP-3,just-RB-5 not-RB-13 penning-VBG-14,NaN,NaN,NaN,NaN,"{'dep_parse': 'prep_at(penning-14, 19-2)\nnsub..."
8,8,0,17,At 19 I was just looking for my next college b...,{I},just not {penning} <<the great English {classi...,next college {boyfriend},SV(O)P,"(nsubj, verb, (O)prep)",I-PRP-3,just-RB-5 not-RB-13 penning-VBG-14,NaN,NaN,NaN,NaN,"{'dep_parse': 'prep_at(penning-14, 19-2)\nnsub..."
9,9,0,19,"Otherwise, I cannot think of anything to admir...",reading {history},Otherwise apart where actually {noted},EVERY SINGLE PAGE {NUMBER},SVO,"(nsubj, verb, dobj)",reading-NN-26 history-NN-27,Otherwise-RB-1 apart-RB-14 where-WRB-28 actual...,EVERY-DT-32 SINGLE-JJ-33 PAGE-NNP-34 NUMBER-NN...,NaN,REASON: prep_in TEXT: this {book} --,NaN,"{'dep_parse': ""advmod(noted-31, Otherwise-1)\n..."


In [28]:
rels_final = []
rels_simp_final = []


'''
Post o sentence o 2 bar naram : check with prev post & index
baghie mokhalafato ham ezafe konam.
'''
prev_post_num = 0
prev_sentence_num = 0
for ind, row in df_rels.iterrows():
    current_post_num = row["post_num"]
    current_sentence_num = row["sentence_num"]   
    if ind > 0:
        if prev_post_num == current_post_num and prev_sentence_num == current_sentence_num:
            #we already extracted the relations of this sentence. Skip it
            continue
        
    #list_rels_final.append(row.tolist())
    try:
        t_annotated = df_rels.iloc[ind]["annotation"]
        t_annotated = ast.literal_eval(t_annotated)
        g_dir = create_dep_graph(t_annotated)
        if g_dir is None:
            print "No extraction found"
            continue
        #g_undir = g_dir.to_undirected()
    except:
        print "Unexpected error while extracting relations:", sys.exc_info()[0]
        continue    
    
    rels_pure, rels_simp = get_relations(g_dir, t_annotated, EXTRACT_NESTED_PREPOSITIONS_RELS=True, option="SVO") 
    #print len(rels_pure)
    for r in rels_pure:
        new_row = row.copy()
        for key in r:
            new_row[key] = r[key]

        #new_row["type"] = "SVO-conjAND"
        #new_row["pattern"] = "(nsubj, verb, object) with conj_and"
        #new_row["arg1"] = r["arg1"]
        #new_row["rel"] = r["rel"]
        #new_row["arg2"] = r["arg2"]
        rels_final.append(new_row.tolist())    

    
    prev_post_num = current_post_num
    prev_sentence_num = current_sentence_num
    
df = pd.DataFrame(rels_final,columns = df_rels.columns.tolist())
#df
df.to_csv(based_dir + "with_conj_and_rels/" + input_name_prefix + file_output_postfix,
          sep=',', encoding='utf-8',header=True, columns=df.columns.tolist(), index=False)

involving-VBG-43  does not appeared as a separate node in parsing tree.
including-VBG-18  does not appeared as a separate node in parsing tree.
following-VBG-3  does not appeared as a separate node in parsing tree.
following-VBG-3  does not appeared as a separate node in parsing tree.
based-VBN-13  does not appeared as a separate node in parsing tree.
following-VBG-5  does not appeared as a separate node in parsing tree.
based-VBN-14  does not appeared as a separate node in parsing tree.
including-VBG-13  does not appeared as a separate node in parsing tree.
Concerning-VBG-1  does not appeared as a separate node in parsing tree.
including-VBG-21  does not appeared as a separate node in parsing tree.
following-VBG-15  does not appeared as a separate node in parsing tree.
based-VBN-6  does not appeared as a separate node in parsing tree.
considering-VBG-25  does not appeared as a separate node in parsing tree.
Following-VBG-1  does not appeared as a separate node in parsing tree.
includi

In [29]:
df2 = df[df["type"].str.contains("conj")]

In [30]:
df2

,Unnamed: 0,post_num,sentence_num,sentence,arg1,rel,arg2,type,pattern,arg1_with_pos,rel_with_pos,arg2_with_pos,arg1_prepositions,rel_prepositions,arg2_prepositions,annotation
10,9,0,19,"Otherwise, I cannot think of anything to admir...",my reading {history},mentally {counted},EVERY SINGLE PAGE {NUMBER},SVO_conj_and_rel,"(nsubj, verb, dobj)",my-PRP$-25 reading-NN-26 history-NN-27,Otherwise-RB-1 apart-RB-14 where-WRB-28 actual...,EVERY-DT-32 SINGLE-JJ-33 PAGE-NNP-34 NUMBER-NN...,,REASON: prep_down TEXT: the {time} --,,"{'dep_parse': ""advmod(noted-31, Otherwise-1)\n..."
14,9,0,19,"Otherwise, I cannot think of anything to admir...",the only {book},mentally {counted},EVERY SINGLE PAGE {NUMBER},SVO_conj_and_rel,"(nsubj, verb, dobj)",the-DT-21 only-JJ-22 book-NN-23,Otherwise-RB-1 apart-RB-14 where-WRB-28 actual...,EVERY-DT-32 SINGLE-JJ-33 PAGE-NNP-34 NUMBER-NN...,,REASON: prep_down TEXT: the {time} --,,"{'dep_parse': ""advmod(noted-31, Otherwise-1)\n..."
18,9,0,19,"Otherwise, I cannot think of anything to admir...",{anything},mentally {counted},EVERY SINGLE PAGE {NUMBER},SVO_conj_and_rel,"(nsubj, verb, dobj)",anything-NN-7,Otherwise-RB-1 apart-RB-14 where-WRB-28 actual...,EVERY-DT-32 SINGLE-JJ-33 PAGE-NNP-34 NUMBER-NN...,,REASON: prep_down TEXT: the {time} --,,"{'dep_parse': ""advmod(noted-31, Otherwise-1)\n..."
22,9,0,19,"Otherwise, I cannot think of anything to admir...",{I},mentally {counted},EVERY SINGLE PAGE {NUMBER},SVO_conj_and_rel,"(nsubj, verb, dobj)",I-PRP-3,Otherwise-RB-1 apart-RB-14 where-WRB-28 actual...,EVERY-DT-32 SINGLE-JJ-33 PAGE-NNP-34 NUMBER-NN...,,REASON: prep_down TEXT: the {time} --,,"{'dep_parse': ""advmod(noted-31, Otherwise-1)\n..."
26,9,0,19,"Otherwise, I cannot think of anything to admir...",{it},mentally {counted},EVERY SINGLE PAGE {NUMBER},SVO_conj_and_rel,"(nsubj, verb, dobj)",it-PRP-19,Otherwise-RB-1 apart-RB-14 where-WRB-28 actual...,EVERY-DT-32 SINGLE-JJ-33 PAGE-NNP-34 NUMBER-NN...,,REASON: prep_down TEXT: the {time} --,,"{'dep_parse': ""advmod(noted-31, Otherwise-1)\n..."
30,9,0,19,"Otherwise, I cannot think of anything to admir...",{I},mentally {counted},EVERY SINGLE PAGE {NUMBER},SVO_conj_and_rel,"(nsubj, verb, dobj)",I-PRP-29,Otherwise-RB-1 apart-RB-14 where-WRB-28 actual...,EVERY-DT-32 SINGLE-JJ-33 PAGE-NNP-34 NUMBER-NN...,,REASON: prep_down TEXT: the {time} --,,"{'dep_parse': ""advmod(noted-31, Otherwise-1)\n..."
45,32,0,41,ade drinking trolls who have gotten their big ...,{cannot},have {gotten},their big girlbig boy {panties},SVO_conj_and_arg1,"(nsubj, verb, dobj)",ade-NN-1 drinking-NN-2 trolls-NNS-3,have-VBP-5 gotten-VBN-6,their-PRP$-7 big-JJ-8 girlbig-JJ-9 boy-NN-10 p...,,,,"{'dep_parse': 'nn(trolls-3, ade-1)\nnn(trolls-..."
85,71,9,36,"As human beings, we see our lives as a narrati...",{we},{look},{ourselves},SVO_conj_and_rel,"(nsubj, verb, dobj)",we-PRP-5,see-VBP-6,ourselves-PRP-13,,,REASON: prep_as TEXT: the {hero} --,"{'dep_parse': 'amod(beings-3, human-2)\nprep_a..."
88,71,9,36,"As human beings, we see our lives as a narrati...",{we},{look},our {lives},SVO_conj_and_rel,"(nsubj, verb, dobj)",we-PRP-5,see-VBP-6,our-PRP$-7 lives-NNS-8,,,REASON: prep_as TEXT: a {narrative} --,"{'dep_parse': 'amod(beings-3, human-2)\nprep_a..."
91,75,9,40,We see the monster's pain and suffering and on...,{We},{suffering},monster {pain},SVO_conj_and_rel,"(nsubj, verb, dobj)",We-PRP-1,see-VBP-2,monster-NN-4 pain-NN-6,,,,"{'dep_parse': 'nsubj(see-2, We-1)\nccomp(resul..."
